# Mongodb

Dans ce TP nous allons utiliser la librairie pymongo, pour créer une base de données dans mongodb avec python puis la questionner.

### Data
Voici des données au format csv, elles sont une partie d'un dataset recensant les crimes au Etats-Unis entre 1984 et 2014.

https://drive.google.com/file/d/10z7kUXDO4BHffJ6ZfVc42CgIs5558vGd/view?usp=sharing

### Création de la BDD

In [4]:
import pandas as pd
from pymongo import MongoClient
import json
import streamlit as st

Créez une fonction python pour passer ce csv dans une base de données mongo, chaque lignes devra être un document.

### Requêter la BDD

Créez un connecteur pour votre BDD

In [45]:
client = MongoClient("localhost", 4344)
db = client.crimes
table = db.documents

In [43]:
# Lecture du csv
df = pd.read_csv('Crime.csv')

In [46]:
# En une seule fois
if table.count_documents({}) == 0:    
    records = df.to_dict(orient='records')
    table.insert_many(records)

In [47]:
# Ligne par ligne
if table.count_documents({}) == 0:
    json_docs = df.apply(lambda x: x.to_json(), axis=1)
    for document in json_docs:

        json_doc = json.loads(document)
        table.insert_one(json_doc)

##### Quelles sont les armes utilisées par les criminel?
Retournez par rapport à tous les documents les valeurs uniques de la clef weapon.

In [8]:
table.distinct("Weapon")

list(table.aggregate([
    { "$group": {
        "_id": "$Weapon", "count" : {"$sum": 1}
    }},
    { "$sort": {"count": -1}}
]))

[{'_id': 'Handgun', 'count': 45748},
 {'_id': 'Knife', 'count': 19586},
 {'_id': 'Blunt Object', 'count': 11150},
 {'_id': 'Shotgun', 'count': 7419},
 {'_id': 'Rifle', 'count': 4968},
 {'_id': 'Firearm', 'count': 3629},
 {'_id': 'Unknown', 'count': 3134},
 {'_id': 'Strangulation', 'count': 1742},
 {'_id': 'Fire', 'count': 1254},
 {'_id': 'Suffocation', 'count': 600},
 {'_id': 'Drowning', 'count': 265},
 {'_id': 'Gun', 'count': 212},
 {'_id': 'Drugs', 'count': 101},
 {'_id': 'Poison', 'count': 80},
 {'_id': 'Explosives', 'count': 61},
 {'_id': 'Fall', 'count': 50}]

##### Combien de crimes ont été commis en 1980?

In [54]:
table.count_documents({"Year":1980})

23092

##### Combien de crimes ont été commis par des hommes au Texas?

In [84]:
table.count_documents({"Perpetrator Sex": "Male", "State": "Texas"})

6160

##### Combien de crimes ont été commis par chaque sexe en Alaska?

In [114]:
maleAndFemalePerpetratorsInAlaska = table.aggregate([
    { "$match": { "State": "Alaska" }},
    { "$group": {"_id": "$Perpetrator Sex", "crimes commis": {"$sum": 1}}},
])

list(maleAndFemalePerpetratorsInAlaska)

[{'_id': 'Male', 'crimes commis': 226},
 {'_id': 'Unknown', 'crimes commis': 63},
 {'_id': 'Female', 'crimes commis': 28}]

##### Combien y a-t'il eu de victimes dans chaque état?

In [19]:
victimInEachState = table.aggregate([
    { "$group" : {
        "_id": "$State", "victimes": {"$sum": "$Victim Count"}
    }},
    { "$sort": { "victimes": -1}}
])

list(victimInEachState)

[{'_id': 'California', 'victimes': 1678},
 {'_id': 'New York', 'victimes': 1182},
 {'_id': 'Florida', 'victimes': 888},
 {'_id': 'Texas', 'victimes': 810},
 {'_id': 'New Jersey', 'victimes': 558},
 {'_id': 'Michigan', 'victimes': 546},
 {'_id': 'Illinois', 'victimes': 512},
 {'_id': 'Pennsylvania', 'victimes': 420},
 {'_id': 'Ohio', 'victimes': 354},
 {'_id': 'Maryland', 'victimes': 256},
 {'_id': 'Massachusetts', 'victimes': 256},
 {'_id': 'Missouri', 'victimes': 250},
 {'_id': 'Louisiana', 'victimes': 230},
 {'_id': 'Washington', 'victimes': 216},
 {'_id': 'Connecticut', 'victimes': 198},
 {'_id': 'Alaska', 'victimes': 188},
 {'_id': 'Indiana', 'victimes': 186},
 {'_id': 'Tennessee', 'victimes': 154},
 {'_id': 'Virginia', 'victimes': 148},
 {'_id': 'Nevada', 'victimes': 132},
 {'_id': 'Arizona', 'victimes': 126},
 {'_id': 'Kentucky', 'victimes': 122},
 {'_id': 'Kansas', 'victimes': 120},
 {'_id': 'North Carolina', 'victimes': 116},
 {'_id': 'Arkansas', 'victimes': 114},
 {'_id': 'Ala

### Bonus

Installez la librairie streamlit
Créez un dashboard qui donne la posibilité de séléctionner une année pour retourner un barplot du nombre de crime commis avec chaque arme.

In [9]:
victimWithEachWeapon = table.aggregate([
    {"$match": {"Year": 1980}},
    { "$group" : {
        "_id": "$Weapon", "victimes": {"$sum": "$Victim Count"}
    }}
])
    
list(victimWithEachWeapon)

[{'_id': 'Drugs', 'victimes': 0},
 {'_id': 'Gun', 'victimes': 12},
 {'_id': 'Rifle', 'victimes': 204},
 {'_id': 'Strangulation', 'victimes': 38},
 {'_id': 'Knife', 'victimes': 204},
 {'_id': 'Unknown', 'victimes': 122},
 {'_id': 'Shotgun', 'victimes': 154},
 {'_id': 'Fall', 'victimes': 2},
 {'_id': 'Blunt Object', 'victimes': 80},
 {'_id': 'Fire', 'victimes': 524},
 {'_id': 'Handgun', 'victimes': 896},
 {'_id': 'Explosives', 'victimes': 32},
 {'_id': 'Poison', 'victimes': 2},
 {'_id': 'Suffocation', 'victimes': 20},
 {'_id': 'Firearm', 'victimes': 116},
 {'_id': 'Drowning', 'victimes': 4}]

In [47]:
# QCM 2
victimWithEachAge = table.aggregate([
    { "$group" : {
        "_id": "$Victim Age", "victimes": {"$sum": 1}
    }},
    { "$sort" : {
        "victimes": -1
    }}
])
    
list(victimWithEachAge)



[{'_id': 25, 'victimes': 3942},
 {'_id': 23, 'victimes': 3581},
 {'_id': 24, 'victimes': 3471},
 {'_id': 22, 'victimes': 3418},
 {'_id': 26, 'victimes': 3397},
 {'_id': 27, 'victimes': 3300},
 {'_id': 30, 'victimes': 3294},
 {'_id': 21, 'victimes': 3193},
 {'_id': 20, 'victimes': 3176},
 {'_id': 29, 'victimes': 3106},
 {'_id': 28, 'victimes': 3064},
 {'_id': 32, 'victimes': 2711},
 {'_id': 19, 'victimes': 2675},
 {'_id': 31, 'victimes': 2664},
 {'_id': 33, 'victimes': 2377},
 {'_id': 34, 'victimes': 2290},
 {'_id': 35, 'victimes': 2275},
 {'_id': 18, 'victimes': 2172},
 {'_id': 36, 'victimes': 1906},
 {'_id': 37, 'victimes': 1853},
 {'_id': 40, 'victimes': 1770},
 {'_id': 17, 'victimes': 1685},
 {'_id': 38, 'victimes': 1675},
 {'_id': 39, 'victimes': 1644},
 {'_id': 99, 'victimes': 1622},
 {'_id': 42, 'victimes': 1468},
 {'_id': 41, 'victimes': 1379},
 {'_id': 43, 'victimes': 1217},
 {'_id': 45, 'victimes': 1195},
 {'_id': 50, 'victimes': 1156},
 {'_id': 44, 'victimes': 1115},
 {'_id':

In [48]:
# QCM 3
victimWithEachAgeByVictim = table.aggregate([
    {"$match": { "Weapon": "Drowning"}},
    { "$group" : {
        "_id": "$Victim Age", "victimes": {"$sum": 1}
    }},
    { "$sort" : {
        "victimes": -1
    }}
])
    
list(victimWithEachAgeByVictim)

[{'_id': 0, 'victimes': 37},
 {'_id': 2, 'victimes': 14},
 {'_id': 1, 'victimes': 12},
 {'_id': 3, 'victimes': 12},
 {'_id': 5, 'victimes': 10},
 {'_id': 24, 'victimes': 9},
 {'_id': 99, 'victimes': 7},
 {'_id': 19, 'victimes': 7},
 {'_id': 36, 'victimes': 7},
 {'_id': 17, 'victimes': 7},
 {'_id': 30, 'victimes': 7},
 {'_id': 4, 'victimes': 6},
 {'_id': 16, 'victimes': 6},
 {'_id': 20, 'victimes': 6},
 {'_id': 22, 'victimes': 6},
 {'_id': 33, 'victimes': 5},
 {'_id': 27, 'victimes': 5},
 {'_id': 7, 'victimes': 5},
 {'_id': 37, 'victimes': 4},
 {'_id': 31, 'victimes': 4},
 {'_id': 38, 'victimes': 4},
 {'_id': 9, 'victimes': 4},
 {'_id': 35, 'victimes': 4},
 {'_id': 26, 'victimes': 4},
 {'_id': 25, 'victimes': 3},
 {'_id': 18, 'victimes': 3},
 {'_id': 28, 'victimes': 3},
 {'_id': 6, 'victimes': 3},
 {'_id': 58, 'victimes': 3},
 {'_id': 41, 'victimes': 3},
 {'_id': 11, 'victimes': 3},
 {'_id': 29, 'victimes': 3},
 {'_id': 46, 'victimes': 3},
 {'_id': 8, 'victimes': 2},
 {'_id': 13, 'victi

In [50]:
# QCM 4
victimWithEachAgeByVictim = table.aggregate([
    {"$match": { "Weapon": "Fall"}},
    { "$group" : {
        "_id": "$State", "victimes": {"$sum": 1}
    }},
    { "$sort" : {
        "victimes": -1
    }}
])
    
list(victimWithEachAgeByVictim)

[{'_id': 'New York', 'victimes': 13},
 {'_id': 'Illinois', 'victimes': 8},
 {'_id': 'Michigan', 'victimes': 3},
 {'_id': 'District of Columbia', 'victimes': 3},
 {'_id': 'North Carolina', 'victimes': 3},
 {'_id': 'Connecticut', 'victimes': 3},
 {'_id': 'Oregon', 'victimes': 2},
 {'_id': 'Rhodes Island', 'victimes': 1},
 {'_id': 'Minnesota', 'victimes': 1},
 {'_id': 'Hawaii', 'victimes': 1},
 {'_id': 'Virginia', 'victimes': 1},
 {'_id': 'South Carolina', 'victimes': 1},
 {'_id': 'Missouri', 'victimes': 1},
 {'_id': 'Massachusetts', 'victimes': 1},
 {'_id': 'Delaware', 'victimes': 1},
 {'_id': 'Ohio', 'victimes': 1},
 {'_id': 'Colorado', 'victimes': 1},
 {'_id': 'Indiana', 'victimes': 1},
 {'_id': 'Alaska', 'victimes': 1},
 {'_id': 'Oklahoma', 'victimes': 1},
 {'_id': 'New Jersey', 'victimes': 1},
 {'_id': 'Florida', 'victimes': 1}]

In [52]:
# QCM 4
victimWithEachAgeByVictim = table.aggregate([
    { "$group" : {
        "_id": "$State", "Moyenne": {"$avg": "$Victim Count"}
    }},
    { "$sort" : {
        "Moyenne": -1
    }}
])
    
list(victimWithEachAgeByVictim)

[{'_id': 'Alaska', 'Moyenne': 0.5930599369085173},
 {'_id': 'North Dakota', 'Moyenne': 0.5172413793103449},
 {'_id': 'Iowa', 'Moyenne': 0.3003003003003003},
 {'_id': 'Connecticut', 'Moyenne': 0.27385892116182575},
 {'_id': 'Washington', 'Moyenne': 0.25471698113207547},
 {'_id': 'Massachusetts', 'Moyenne': 0.2471042471042471},
 {'_id': 'Maine', 'Moyenne': 0.24489795918367346},
 {'_id': 'New Jersey', 'Moyenne': 0.23134328358208955},
 {'_id': 'Minnesota', 'Moyenne': 0.22171945701357465},
 {'_id': 'Vermont', 'Moyenne': 0.20689655172413793},
 {'_id': 'Montana', 'Moyenne': 0.2037037037037037},
 {'_id': 'Nevada', 'Moyenne': 0.2006079027355623},
 {'_id': 'Nebraska', 'Moyenne': 0.1693548387096774},
 {'_id': 'Kansas', 'Moyenne': 0.16806722689075632},
 {'_id': 'Wisconsin', 'Moyenne': 0.16666666666666666},
 {'_id': 'Idaho', 'Moyenne': 0.16184971098265896},
 {'_id': 'Wyoming', 'Moyenne': 0.16129032258064516},
 {'_id': 'Oregon', 'Moyenne': 0.14501510574018128},
 {'_id': 'Florida', 'Moyenne': 0.12407